# Python script for earthquakes in South America (up to 22.05.2018)

First select the area and timeline you want, and import the data as a csv-file from: https://earthquake.usgs.gov/earthquakes/search/ 

By choosing **Advanced Options** you are able to specify longitude and latitude and choose other factors as you please. In the Advanced Options, choose **Event type as: Earthquake**. 

*The data chosen here is limited to magnitude 4.5>
And between latitude: -55.361 to 7.995
And longitude: -84.982 to -60.175* 

As **Output Options choose: CSV**. You may select the order of data by time from most recent to oldest or the other way around, or by magnitude from strongest to weakest or the other way around.

Once you have made your selections, press: **Search. Save the opened page as a file**.

*There is no need to try to clean the file, although it is relatively difficult to read in Excel or TextWrangler.* 
When reading the data into Jupyter or Spyder, use **Pandas**, as the file includes also lots of text. 

#### First import modules:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Import additional modules:**

*Import **Basemap** in order to plot onto a map. First open a new Terminal window, and type: conda install Basemap. After installation you can import it.*

*Import **colors** for scatter plots and colorbar.*

*Import **cm** for cmap.*

*Import **inset_axes** for positioning a background box for the legend.*

*Import **Rectangle** for making a background box for the legend.*

In [2]:
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib import colors
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle

#### Set path to data:

In [3]:
fp = "/Users/Nellimetiainen/Desktop/SA-earthquakes.csv"

#### Set dataframe and datetime index:
**Read it in as a csv-file, parse dates, determine the index column as time and separate columns with a comma.** 

In [4]:
dataFrame = pd.read_csv(fp, parse_dates=True, index_col='time', sep=',')

## Checking and testing data 
(Not neccessary)

Check column names to select useful data:

In [5]:
dataFrame.columns

Index(['latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap',
       'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

Check datatypes to make sure everything is as it should be:

In [6]:
dataFrame.dtypes

latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

Find latitude and longitude bounds for earthquake data:

In [7]:
print("Minimum latitude:", dataFrame['latitude'].min())
print("Maximum latitude:", dataFrame['latitude'].max())
print("Minimum longitude:", dataFrame['longitude'].min())
print("Maximum longitude:", dataFrame['longitude'].max())

Minimum latitude: -55.361
Maximum latitude: 7.995
Minimum longitude: -84.982
Maximum longitude: -60.175


Find minimum and maximum magnitude and depth:

In [8]:
print("Minimum magnitude:", dataFrame['mag'].min())
print("Maximum magnitude:", dataFrame['mag'].max())
print("Minumum depth:", dataFrame['depth'].min())
print("Maximum depth:", dataFrame['depth'].max())

Minimum magnitude: 4.5
Maximum magnitude: 9.5
Minumum depth: 0.0
Maximum depth: 650.0


You can find the variables in memory using `%whos`:

In [9]:
%whos

Variable          Type         Data/Info
----------------------------------------
AnchoredSizeBar   type         <class 'mpl_toolkits.axes<...>artists.AnchoredSizeBar'>
Basemap           type         <class 'mpl_toolkits.basemap.Basemap'>
Rectangle         type         <class 'matplotlib.patches.Rectangle'>
cm                module       <module 'matplotlib.cm' f<...>ckages/matplotlib/cm.py'>
colors            module       <module 'matplotlib.color<...>es/matplotlib/colors.py'>
dataFrame         DataFrame                             <...>[18493 rows x 21 columns]
fp                str          /Users/Nellimetiainen/Desktop/SA-earthquakes.csv
inset_axes        function     <function inset_axes at 0x119a542f0>
np                module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
pd                module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
plt               module       <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>


# Plotting data

## Tests 
(First plots are not necessary)

[Test plots](Test plots SA earthquakes.ipynb)

# Make new columns to organize data by value:

#### Separate columns for different depth & magnitude categories

In [13]:
# Depths:
dataFrame['Deep'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 300.01) & (dataFrame['depth'] <= 700.00)]
dataFrame['Intermediate'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 70.01) & (dataFrame['depth'] <= 300.00)]
dataFrame['Shallow'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 0.00) & (dataFrame['depth'] <= 70.00)]

# Magnitudes:
dataFrame['Light'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 4.5) & (dataFrame['mag'] <= 4.9)]
dataFrame['Moderate'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 5.0) & (dataFrame['mag'] <= 5.9)]
dataFrame['Strong'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 6.0) & (dataFrame['mag'] <= 6.9)]
dataFrame['Major'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 7.0) & (dataFrame['mag'] <= 7.9)]
dataFrame['Great'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 8.0) & (dataFrame['mag'] <= 10)]

# All earthquakes with a magnitude of over 6:
dataFrame['Mag over 6'] = dataFrame['mag'].loc[(dataFrame['mag'] >= 6.0) & (dataFrame['mag'] <= 10)]

#### New columns for deepest earthquakes for each 100 m

In [14]:
dataFrame['300'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 300.00) & (dataFrame['depth'] <= 399.99)]
dataFrame['400'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 400.00) & (dataFrame['depth'] <= 499.99)]
dataFrame['500'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 500.00) & (dataFrame['depth'] <= 599.99)]
dataFrame['600'] = dataFrame['depth'].loc[(dataFrame['depth'] >= 600.00) & (dataFrame['depth'] <= 650.00)]

In [15]:
print("Minumum depth:", dataFrame['300'].min())
print("Maximum depth:", dataFrame['300'].max())
print("Minumum depth:", dataFrame['400'].min())
print("Maximum depth:", dataFrame['400'].max())
print("Minumum depth:", dataFrame['500'].min())
print("Maximum depth:", dataFrame['500'].max())
print("Minumum depth:", dataFrame['600'].min())
print("Maximum depth:", dataFrame['600'].max())

Minumum depth: 308.4
Maximum depth: 363.6
Minumum depth: 429.6
Maximum depth: 436.5
Minumum depth: 500.0
Maximum depth: 599.9
Minumum depth: 600.0
Maximum depth: 650.0


#### Check dataframe, that new columns have been registered & that their type is correct 

In [16]:
dataFrame.columns

Index(['latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap',
       'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource', 'Deep', 'Intermediate', 'Shallow',
       'Light', 'Moderate', 'Strong', 'Major', 'Great', 'Mag over 6', '300',
       '400', '500', '600'],
      dtype='object')

In [17]:
dataFrame.dtypes

latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
Deep               float64
Intermediate       float64
Shallow            float64
Light              float64
Moderate           float64
Strong             float64
Major              float64
Great              float64
Mag over 6         float64
300                float64
400                float64
500                float64
600                float64
dtype: object

# MAIN CODE

Make plots for each month of each year showing the earthquake magnitudes on scatter plot and depths and amount as barplot. Loop earthquakes for each year.

In [ ]:
# Set figure size:
plt.rcParams['figure.figsize'] = [12,12]
# Set path for saving images:
savepath="/Users/Nellimetiainen/Desktop/a/"

#### Set up for bar plot:

# Make a bar every 2 degrees latitude:
barIncrement = 2
barBins= np.arange(8, -56, -barIncrement)
# Determine the length of bars:
numBars = len(barBins)
# Separate different depth earthquakes:
shallowCount = np.zeros(numBars)
intermediateCount = np.zeros(numBars)
deepCount = np.zeros(numBars)

# For a running value text box with the month and year displayed, assign names for values for 'months':
months = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# Select time range: first line years, second line months:
# Loop data  
for year in range(1960, 2018):
    for month in range(1, 13):
        currentMonth = str(month)+"-"+str(year)
        selectedMonth = dataFrame[currentMonth]
        
# Set up figure for subplots:
        f, ax = plt.subplots(nrows=1, ncols=2)

# Title for whole figure:
        plt.suptitle('Earthquakes in South America 1960-2017', fontweight='bold')

# Make bar plot (set axis, load values):
        for i in range(numBars):
            numShallow = selectedMonth['Shallow'].loc[(selectedMonth['latitude'] > barBins[i]-barIncrement) & (selectedMonth['latitude'] <= barBins[i])].count()
            shallowCount[i] = shallowCount[i] + numShallow
            
            numIntermediate = selectedMonth['Intermediate'].loc[(selectedMonth['latitude'] > barBins[i]-barIncrement) & (selectedMonth['latitude'] <= barBins[i])].count()
            intermediateCount[i] = intermediateCount[i] + numIntermediate
            
            numDeep = selectedMonth['Deep'].loc[(selectedMonth['latitude'] > barBins[i]-barIncrement) & (selectedMonth['latitude'] <= barBins[i])].count()
            deepCount[i] = deepCount[i] + numDeep
# Plot bars:
        ax[0].barh(y=barBins-(barIncrement/2), width=shallowCount, label='0 - 70 km', color='r', alpha = 0.4)
        ax[0].barh(y=barBins-(barIncrement/2), width=intermediateCount, left=shallowCount, label='70 - 300 km', color='g', alpha = 0.4) 
        ax[0].barh(y=barBins-(barIncrement/2), width=deepCount, left=shallowCount+intermediateCount, label='300 - 670 km', color='b', alpha = 0.4)

# Determine total count and scale limits:
        totalCount = shallowCount + intermediateCount + deepCount
        fewQuakes = 50
        someQuakes = 100
        intQuakes = 200
        medQuakes = 500
        manyQuakes = 1000
        allQuakes = 1700
# Determine limits & scalejumps & indicate scalejumps with changing x_label color if necessary (here all black):
        if totalCount.max() < fewQuakes:
            ax[0].set_xlim(fewQuakes,0)
            ax[0].set_xticks(np.arange(0,fewQuakes+1,10))
            ax[0].set_xlabel('Number of earthquakes', color='black')
        elif totalCount.max() < someQuakes:
            ax[0].set_xlim(someQuakes,0)
            ax[0].set_xticks(np.arange(0,someQuakes+1,20))
            ax[0].set_xlabel('Number of earthquakes', color='black')
        elif totalCount.max() < intQuakes:
            ax[0].set_xlim(intQuakes,0)
            ax[0].set_xticks(np.arange(0,intQuakes+1,20))
            ax[0].set_xlabel('Number of earthquakes', color='black')
        elif totalCount.max() < medQuakes:
            ax[0].set_xlim(medQuakes,0)
            ax[0].set_xticks(np.arange(0,medQuakes+1,50))
            ax[0].set_xlabel('Number of earthquakes', color='black')
        elif totalCount.max() < manyQuakes:
            ax[0].set_xlim(manyQuakes,0)
            ax[0].set_xticks(np.arange(0,manyQuakes+1,100))
            ax[0].set_xlabel('Number of earthquakes', color='black')
        else:
            ax[0].set_xlim(allQuakes, 0)
            ax[0].set_xticks(np.arange(0,allQuakes+1,200))
            ax[0].set_xlabel('Number of earthquakes', color='black')

# Add grid:
        ax[0].grid(axis='x')

# Add titles for bar plot:
        ax[0].set_title('Amount of earthquakes according to depth', color='blue')
        ax[0].set_ylabel('Latitude', color='blue')

# Add legend and assign location:
        ax[0].legend(loc = 3, title='Depth range')

#### Make map figure on the right side of the barplot to plot earthquake locations and magnitudes:
# Setup Lambert Conformal basemap:
# Set resolution=None to skip processing of boundary datasets:
        m = Basemap(width=4000000,height=9000000,projection='cyl',
            resolution='c',lat_1=-20,lat_2=-30,lat_0=-25,lon_0=-75,llcrnrlat=-58,urcrnrlat=10,
            llcrnrlon=-90,urcrnrlon=-60)

# Add borders or shoreline (take off #):
        #m.drawcountries()
        #m.drawcoastlines()

# Import topography map - to change colour intensity set alpha to wanted level:
        m.arcgisimage(service='World_Physical_Map')

# Set axis:
        lon,lat = m(selectedMonth["longitude"].values, selectedMonth["latitude"].values)

# Make scatter plot. s=size of points, c=color, cmap determines chosen colorbar colors, alpha makes points less opaque, edgecolors determines the color of point edges:    
        plot1 = plt.scatter(lon, lat, s=0.0001*selectedMonth["mag"]**8, c=selectedMonth["mag"], cmap='plasma_r', alpha=0.5, edgecolors='black')

# Set grid for parallels and meridians:
        x, y = m(*np.meshgrid(lon,lat))
        parallels = np.arange(-80.,81.,10.)
        meridians = np.arange(10.,351.,10.)

# Set labels [left,right,top,bottom] for parallels and meridians:
        m.drawparallels(parallels,labels=[True,False,True,True])
        m.drawmeridians(meridians,labels=[False,False,False,True])

# Add titles for whole plot and x & y:
        ax[1].set_title('Earthquake locations and magnitudes in {0:d}'.format(year), color='blue')
        ax[1].set_xlabel('Longitude', color='blue')
        ax[1].set_ylabel('Latitude', color='blue')
   
# Specify colors:
        cmap = cm.get_cmap('plasma', 7)
# Create bounds for colorbar (min and max values for magnitude, and the number of separate values displayed):
        bounds = np.linspace(4, 10, 7)
        plot1.set_clim(4.5, 9.5)
# Make a white box background (determine height and width, location and padding at edges:
# Assign spines and ticks, choose color:
        cbbox = inset_axes(ax[1], '27%', '35%', loc = 6, borderpad=0.9)
        [cbbox.spines[k].set_visible(False) for k in cbbox.spines]
        cbbox.tick_params(axis='both', left='off', top='off', right='off', bottom='off', labelleft='off', labeltop='off', labelright='off', labelbottom='off')
        cbbox.set_facecolor([1,1,1,0.7])
# Set axins:
        axins1 = inset_axes(ax[1], width="10%", height="30%", loc=6, borderpad=2)
# Create colorbar:
        cbar = plt.colorbar(cax=axins1, cmap=cmap, orientation="vertical", boundaries=bounds)
# Add colorbar label:
        cbar.set_label('Magnitude', color='blue')


#### Not neccessary: #######################################################
# Make a scale bar for the map:
        #scalebar = AnchoredSizeBar(ax[1].transData,
                           #5, 'distance', 'lower left', 
                           #pad=1,
                           #color='black',
                           #frameon=False,
                           #size_vertical=0.2)

        #ax[1].add_artist(scalebar)

# Make a North arrow for map:
       #plot1.annotate('N', xy = (-85, -40), xytext=(-86.5, -50), color='r', size=30, arrowprops=dict(arrowstyle="fancy", color='r'))

##########################################################################

# Print month and year onto figure into a color box:
# Note: We first need to convert the lon, lat to the projected values:
        textLon, textLat = m(-88.5, -38)
        ax[1].text(textLon, textLat, months[month]+" "+str(year), fontsize=15,
        name='Courier New',
        bbox={'facecolor':'white', 'edgecolor':'none', 'alpha':0.7, 'pad':5})

# Make plots fill the whole figure:
       #plt.tight_layout(w_pad=10)

# Make space between main title and plots:
        plt.subplots_adjust(top=0.94)

# Save figure as picture and assign where to save:
        plotFile = str(year) + "-" + str(month) + ".png"
        plt.savefig(savepath+plotFile)

# Save memory and avoid computer crashing by not opening all the figures in the loop:
        plt.close("all")

# CROSS SECTION VERSIONS

[Cross section version](Cross section versions SA earthquakes.ipynb)

# Other versions:

[All earthquakes](All earthquakes SA.ipynb)

[By depth](By depth.ipynb)

[By magnitude](SA earthquakes by magnitude.ipynb)

## Short cut (for testing changes on map)

[Testing version for map](Testing version for SA eq map.ipynb)